<a href="https://colab.research.google.com/github/Rhitabrat/deep-learning-CS510/blob/main/hw3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW 3

In [30]:
import tensorflow as tf 
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
import numpy as np

In [31]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
assert X_train.shape == (60000, 28, 28)
assert X_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [32]:
# expand new axis, channel axis 
x_train = np.expand_dims(X_train, axis=-1)

# [optional]: we may need 3 channel (instead of 1)
x_train = np.repeat(x_train, 3, axis=-1)

# it's always better to normalize 
x_train = x_train.astype('float32') / 255

# resize the input shape , i.e. old shape: 28, new shape: 32
x_train = tf.image.resize(x_train, [32, 32])

# one hot 
y_train = tf.keras.utils.to_categorical(y_train , num_classes=10)

print(x_train.shape, y_train.shape)

(60000, 32, 32, 3) (60000, 10)


In [33]:
model = ResNet50(weights="imagenet",
                #  input_shape=(224, 224, 3),
                 include_top=False)

model.trainable = False

In [40]:
inputs = keras.Input(shape=(32, 32, 3))

# Separately from setting trainable on the model, we set training to False 
x = model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(10, activation="relu")(x)

model = keras.Model(inputs, outputs)

In [41]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 model_5 (Functional)        (None, 1, 1, 10)          23608202  
                                                                 
 global_average_pooling2d_1   (None, 10)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
Total params: 23,608,312
Trainable params: 20,600
Non-trainable params: 23,587,712
_________________________________________________________________


In [49]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.CategoricalCrossentropy(name="categorial_loss"),
              metrics=["accuracy"])

In [50]:
model.fit(x_train, y_train, 
        #   steps_per_epoch=12, 
          validation_split=0.2,
        #   validation_steps=4, 
          epochs=5)

Epoch 1/5
1500/1500 [==============================] - 27s 16ms/step - loss: 9.9320 - accuracy: 0.2516 - val_loss: 9.9315 - val_accuracy: 0.2741
Epoch 2/5
1500/1500 [==============================] - 23s 15ms/step - loss: 9.8580 - accuracy: 0.2802 - val_loss: 9.9321 - val_accuracy: 0.2898
Epoch 3/5
1500/1500 [==============================] - 23s 15ms/step - loss: nan - accuracy: 0.2632 - val_loss: 12.8276 - val_accuracy: 0.1060
Epoch 4/5
1500/1500 [==============================] - 23s 15ms/step - loss: 12.7650 - accuracy: 0.1140 - val_loss: 12.8162 - val_accuracy: 0.1060
Epoch 5/5
1500/1500 [==============================] - 23s 15ms/step - loss: 12.7626 - accuracy: 0.1140 - val_loss: 12.8151 - val_accuracy: 0.1060
